In [8]:
import sys
sys.path.append('../input/d/workspace/all-submission-10-8/mysitepackages/kaggle/working/mysitepackages')

In [9]:
import warnings
warnings.filterwarnings('ignore')
from collections import OrderedDict
import os
import sys 
import json
import glob
import random
import re
import collections
import time
import torch.nn as nn
import numpy as np
import pandas as pd
import pydicom
import cv2
import matplotlib.pyplot as plt
import seaborn as sns 
sys.path.insert(0, 'furnace')
import torch
from torch import nn
from torch.utils import data as torch_data
from sklearn import model_selection as sk_model_selection
from torch.nn import functional as torch_functional

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

data_directory = '../input/rsna-miccai-brain-tumor-radiogenomic-classification'
pytorch3dpath = "../input/efficientnetpyttorch3d/EfficientNet-PyTorch-3D"
monaipath = "../input/rsna-dataset/monai/monai"
vitpath = "../input/rsna-dataset/vit3d-pytorch-main/vit3d-pytorch-main"

In [ ]:
TEST  = False
if(TEST):  
    mri_types = ['FLAIR']
else:
    mri_types = ['FLAIR', 'T1w', 'T1wCE', 'T2w']
    
DROPOUT_RATE = 0.3
SEED = 42

GPU_NUM = 1
SIZE = 256
NUM_IMAGES = 64
BATCH_SIZE = 4 * GPU_NUM
N_EPOCHS = 8
SEED = 42
LEARNING_RATE = 0.0005
LR_DECAY = 0.9
num_slices = 1
random_slices = False
DROPOUT_RATE = DROPOUT_RATE
MODEL = "ViT3D"
modelfiles = ['../input/d/workspace/all-submission-10-8/model3d3407/model3d3407/ViT3D/FLAIR-e1-loss0.690-auc0.609-FOLD0.pth']


sys.path.append(pytorch3dpath)
sys.path.append(vitpath)
sys.path.append(monaipath)


from efficientnet_pytorch_3d import EfficientNet3D
import monai
from monai.networks.nets.densenet import DenseNet
from monai.networks.nets.densenet import DenseNet121
from monai.networks.nets.densenet import DenseNet169
from monai.networks.nets.densenet import DenseNet201
from monai.networks.nets.densenet import DenseNet264
from monai.networks.nets.densenet import DenseNet264

from monai.networks.nets.resnet import ResNet
from monai.networks.nets.resnet import resnet34
from monai.networks.nets.resnet import resnet50
from monai.networks.nets.resnet import resnet101
from monai.networks.nets.resnet import resnet152
from monai.networks.nets.resnet import resnet200
from monai.networks.nets.senet import SEResNet50
from monai.networks.nets.senet import SEresnet101
from monai.networks.nets.senet import SEresnet152
from monai.networks.nets.senet import SEResNext50
from monai.networks.nets.senet import SEResNext101

from vit3d_pytorch import ViT3D
from vit_pytorch.deepvit import DeepViT
from vit_pytorch.t2t import T2TViT
from vit_pytorch.cct import CCT
from vit_pytorch.cross_vit import CrossViT
from vit_pytorch.pit import PiT
from vit_pytorch.levit import LeViT
from vit_pytorch.cvt import CvT
from vit_pytorch.twins_svt import TwinsSVT
from vit_pytorch.nest import NesT

from vit_pytorch.efficient import ViT
from x_transformers import Encoder

In [10]:
def load_dicom_image(path, img_size=SIZE):
    dicom = pydicom.read_file(path)
    data = dicom.pixel_array
    if np.min(data)==np.max(data):
        data = np.zeros((img_size,img_size))
        return data
    
    data = cv2.resize(data, (img_size, img_size))
    return data


def natural_sort(l): 
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)', key)]
    return sorted(l, key=alphanum_key)


def load_dicom_images_3d(scan_id, num_imgs=NUM_IMAGES, img_size=SIZE, mri_type="FLAIR", split="train"):
    files = natural_sort(glob.glob(f"{data_directory}/{split}/{scan_id}/{mri_type}/*.dcm"))
    
    every_nth = len(files) / num_imgs
    indexes = [min(int(round(i*every_nth)), len(files)-1) for i in range(0,num_imgs)]
    files_to_load = [files[i] for i in indexes]

    if(split=="train" and random_slices):
        if(every_nth>1):
            files_to_load = natural_sort(random.sample(files, num_imgs))

    if(num_slices>1):
        import math
        steps = int(math.floor(num_slices/2.0))
        files_slices_to_load = [[files_to_load[j] for j in range(max(i - steps,0), min(i + steps + 1,NUM_IMAGES))] for i in range(len(indexes))]
        img3d = np.stack([np.mean([load_dicom_image(f) for f in f_list],axis=0) for f_list in files_slices_to_load]).T
    else:
        img3d = np.stack([load_dicom_image(f) for f in files_to_load]).T 
    img3d = img3d - np.min(img3d)
    if np.max(img3d) != 0:
        img3d = img3d / np.max(img3d)
    
    return np.expand_dims(img3d,0)


load_dicom_images_3d("00000", mri_type=mri_types[0]).shape

In [ ]:
def set_seed(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True

set_seed(SEED)


directory = 'model3d'+str(SEED)+'/'+MODEL

if not os.path.exists(directory):
    os.makedirs(directory)


samples_to_exclude = [109, 123, 709]

train_df = pd.read_csv(f"{data_directory}/train_labels.csv")
print("original shape", train_df.shape)
train_df = train_df[~train_df.BraTS21ID.isin(samples_to_exclude)]
print("new shape", train_df.shape)

df_train, df_valid = sk_model_selection.train_test_split(
    train_df, 
    test_size=0.2, 
    random_state=SEED, 
    stratify=train_df["MGMT_value"], 
)

In [ ]:
class Dataset(torch_data.Dataset):
    def __init__(self, paths, targets=None, mri_type=None, split="train"):
        self.paths = paths
        self.targets = targets
        self.mri_type = mri_type
        self.split = split
          
    def __len__(self):
        return len(self.paths)
    
    def __getitem__(self, index):
        scan_id = self.paths[index]
        if self.targets is None:
            data = load_dicom_images_3d(str(scan_id).zfill(5), mri_type=self.mri_type[index], split=self.split)
        else:
            data = load_dicom_images_3d(str(scan_id).zfill(5), mri_type=self.mri_type[index], split="train")
            
        if self.targets is None:
            return {"X": data, "id": scan_id}
        else:
            return {"X": data, "y": torch.tensor(self.targets[index], dtype=torch.float)}


In [ ]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        if(MODEL=='DenseNet'):
            self.net = model = DenseNet(spatial_dims=3, in_channels=1, out_channels=1)
        elif(MODEL=='DenseNet121'):
            self.net = model = DenseNet121(spatial_dims=3, in_channels=1, out_channels=1)
        elif(MODEL=='DenseNet169'):
            self.net = model = DenseNet169(spatial_dims=3, in_channels=1, out_channels=1)
        elif(MODEL=='DenseNet201'):
            self.net = model = DenseNet201(spatial_dims=3, in_channels=1, out_channels=1)
        elif(MODEL=='DenseNet264'):
            self.net = model = DenseNet264(spatial_dims=3, in_channels=1, out_channels=1)
        
        if(MODEL=='resnet34'):
            self.net = model = resnet34(spatial_dims=3,pretrained=False,n_input_channels=1)
        elif(MODEL=='resnet50'):
            self.net = model = resnet50(spatial_dims=3,pretrained=False,n_input_channels=1)
        elif(MODEL=='resnet101'):
            self.net = model = resnet101(spatial_dims=3,pretrained=False,n_input_channels=1)
        elif(MODEL=='resnet152'):
            self.net = model = resnet152(spatial_dims=3,pretrained=False,n_input_channels=1)
        elif(MODEL=='resnet200'):
            self.net = model = resnet200(spatial_dims=3,pretrained=False,n_input_channels=1)

        if(MODEL=='resnet34'):
            self.net = model = resnet34(spatial_dims=3,pretrained=False,n_input_channels=1)
        elif(MODEL=='resnet50'):
            self.net = model = resnet50(spatial_dims=3,pretrained=False,n_input_channels=1)
        elif(MODEL=='resnet101'):
            self.net = model = resnet101(spatial_dims=3,pretrained=False,n_input_channels=1)
        elif(MODEL=='resnet152'):
            self.net = model = resnet152(spatial_dims=3,pretrained=False,n_input_channels=1)
        elif(MODEL=='resnet200'):
            self.net = model = resnet200(spatial_dims=3,pretrained=False,n_input_channels=1)

        if('efficientnet' in MODEL):
            self.net = EfficientNet3D.from_name(MODEL, override_params={'num_classes': 2}, in_channels=1)

        if(MODEL=='SEResNet50'):
            self.net = SEResNet50(spatial_dims=3, in_channels=1)
        elif(MODEL=='SEresnet101'):
            self.net = SEresnet101(spatial_dims=3, in_channels=1)
        elif(MODEL=='SEresnet152'):
            self.net = SEresnet152(spatial_dims=3, in_channels=1)
        elif(MODEL=='SEResNext50'):
            self.net = SEResNext50(spatial_dims=3, in_channels=1)
        elif(MODEL=='SEResNext101'):
            self.net = SEResNext101(spatial_dims=3, in_channels=1)

        if('ViT3D' in MODEL):
            self.net = ViT3D(
            image_size=(256, 256, NUM_IMAGES),
            patch_size=32,
            num_classes=1,
            dim=1024,
            depth=6,
            heads=16,
            mlp_dim=2048,
            dropout=DROPOUT_RATE,
            emb_dropout=DROPOUT_RATE
            )
            print('ViT3D')

        if('SE' in MODEL):
            n_features = self.net.last_linear.in_features
            self.net.last_linear = nn.Sequential(
            OrderedDict(
                [
                    ('dropout1', nn.Dropout(DROPOUT_RATE)),
                    ("out", nn.Linear(in_features=n_features, out_features=1, bias=True)),
                ]
            )
        )
        elif('resnet' in MODEL):
            n_features = self.net.fc.in_features
            self.net.fc = nn.Sequential(
            OrderedDict(
                [
                    ('dropout1', nn.Dropout(DROPOUT_RATE)),
                    ("out", nn.Linear(in_features=n_features, out_features=1, bias=True)),
                ]
            )
        )
        elif('efficientnet' in MODEL):
            n_features = self.net._fc.in_features
            self.net._fc = nn.Sequential(
            OrderedDict(
                [
                    ('dropout1', nn.Dropout(DROPOUT_RATE)),
                    ("out", nn.Linear(in_features=n_features, out_features=1, bias=True)),
                ]
            )
        )
        elif('Dense' in MODEL):
            in_channels = self.net.class_layers[3].in_features
            self.net.class_layers = nn.Sequential(
            OrderedDict(
                [
                    ('dropout1', nn.Dropout(DROPOUT_RATE)),
                    ("relu", self.net.class_layers[0]),
                    ("pool", self.net.class_layers[1]),
                    ("flatten", self.net.class_layers[2]),
                    ("out", self.net.class_layers[3]),
                ]
            )
        )
        bn_eps = 1e-5
        bn_momentum = 0.1
    def forward(self, x):
        out = self.net(x)
        return out
        
def build_model():
    device_ids = [0, 1 , 2]
    if(GPU_NUM==3):
        model = torch.nn.DataParallel(Model(), device_ids=device_ids)
    else:
        model = Model()
    return model    


class Trainer:
    def __init__(
        self, 
        model, 
        device, 
        optimizer, 
        criterion
    ):
        self.model = model
        self.device = device
        self.optimizer = optimizer
        self.lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(self.optimizer, gamma=LR_DECAY)
        self.criterion = criterion

        self.best_valid_score = .0
        self.n_patience = 0
        self.lastmodel = None
        
        self.val_losses = []
        self.train_losses = []
        self.val_auc = []
        
    def fit(self, epochs, train_loader, valid_loader, save_path, patience):      
        for n_epoch in range(1, epochs + 1):
            self.info_message("EPOCH: {}", n_epoch)
            
            train_loss, train_time = self.train_epoch(train_loader)
            valid_loss, valid_auc, valid_time = self.valid_epoch(valid_loader)
            
            self.train_losses.append(train_loss)
            self.val_losses.append(valid_loss)
            self.val_auc.append(valid_auc)
            
            self.info_message(
                "[Epoch Train: {}] loss: {:.4f}, time: {:.2f} s",
                n_epoch, train_loss, train_time
            )
            
            self.info_message(
                "[Epoch Valid: {}] loss: {:.4f}, auc: {:.4f}, time: {:.2f} s",
                n_epoch, valid_loss, valid_auc, valid_time
            )

            if self.best_valid_score < valid_auc: 
                self.save_model(n_epoch, save_path, valid_loss, valid_auc)
                self.info_message(
                     "auc improved from {:.4f} to {:.4f}. Saved model to '{}'", 
                    self.best_valid_score, valid_auc, self.lastmodel
                )
                self.best_valid_score = valid_auc
                self.n_patience = 0
            else:
                self.n_patience += 1
            
            if self.n_patience >= patience:
                self.info_message("\nValid auc didn't improve last {} epochs.", patience)
                break
            
    def train_epoch(self, train_loader):
        self.model.train()
        t = time.time()
        sum_loss = 0
        # print(self.lr_scheduler.get_lr())
        for step, batch in enumerate(train_loader, 1):
            X = batch["X"].float().to(self.device)
            # print(X.shape)
            targets = batch["y"].to(self.device)
            self.optimizer.zero_grad()
            outputs = self.model(X).squeeze(1)
            loss = self.criterion(outputs, targets)
                
            loss.backward()

            sum_loss += loss.detach().item()
            
            self.optimizer.step()
            
            message = 'Train Step {}/{}, train_loss: {:.4f}'
            self.info_message(message, step, len(train_loader), sum_loss/step, end="\r")
        # print(self.lr_scheduler.get_lr())
        self.lr_scheduler.step()
        
        return sum_loss/len(train_loader), int(time.time() - t)
    
    def valid_epoch(self, valid_loader):
        self.model.eval()
        t = time.time()
        sum_loss = 0
        y_all = []
        outputs_all = []

        for step, batch in enumerate(valid_loader, 1):
            with torch.no_grad():
                targets = batch["y"].to(self.device)

                output = torch.sigmoid(self.model(batch["X"].float().to(self.device)).squeeze(1))
                loss = self.criterion(output, targets)
                sum_loss += loss.detach().item()

                y_all.extend(batch["y"].tolist())
                outputs_all.extend(output.tolist())

            message = 'Valid Step {}/{}, valid_loss: {:.4f}'
            self.info_message(message, step, len(valid_loader), sum_loss/step, end="\r")
            
        y_all = [1 if x > 0.5 else 0 for x in y_all]
        auc = roc_auc_score(y_all, outputs_all)
        
        return sum_loss/len(valid_loader), auc, int(time.time() - t)
    
    def save_model(self, n_epoch, save_path, loss, auc):
        self.lastmodel = directory+'/'+f"{save_path}-e{n_epoch}-loss{loss:.3f}-auc{auc:.3f}-FOLD{FOLD}.pth"
        torch.save(
            {
                "model_state_dict": self.model.state_dict(),
                "optimizer_state_dict": self.optimizer.state_dict(),
                "best_valid_score": self.best_valid_score,
                "n_epoch": n_epoch,
            },
            self.lastmodel,
        )
        
    def display_plots(self, mri_type):
        plt.figure(figsize=(10,5))
        plt.title("{}: Training and Validation Loss")
        plt.plot(self.val_losses,label="val")
        plt.plot(self.train_losses,label="train")
        plt.xlabel("iterations")
        plt.ylabel("Loss")
        plt.legend()
        plt.show()
        plt.close()
        
        plt.figure(figsize=(10,5))
        plt.title("{}: Validation AUC-ROC")
        plt.plot(self.val_auc,label="val")
        plt.xlabel("iterations")
        plt.ylabel("AUC")
        plt.legend()
        plt.show()
        plt.close()
    
    @staticmethod
    def info_message(message, *args, end="\n"):
        print(message.format(*args), end=end)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def train_mri_type(df_train, df_valid, mri_type):
    if mri_type=="all":
        train_list = []
        valid_list = []
        for mri_type in mri_types:
            df_train.loc[:,"MRI_Type"] = mri_type
            train_list.append(df_train.copy())
            df_valid.loc[:,"MRI_Type"] = mri_type
            valid_list.append(df_valid.copy())

        df_train = pd.concat(train_list)
        df_valid = pd.concat(valid_list)
    else:
        df_train.loc[:,"MRI_Type"] = mri_type
        df_valid.loc[:,"MRI_Type"] = mri_type

    print(df_train.shape, df_valid.shape)
    
    train_data_retriever = Dataset(
        df_train["BraTS21ID"].values, 
        df_train["MGMT_value"].values, 
        df_train["MRI_Type"].values
    )

    valid_data_retriever = Dataset(
        df_valid["BraTS21ID"].values, 
        df_valid["MGMT_value"].values,
        df_valid["MRI_Type"].values
    )

    train_loader = torch_data.DataLoader(
        train_data_retriever,
        batch_size=BATCH_SIZE ,
        shuffle=True,
        num_workers=8 * GPU_NUM,
    )

    valid_loader = torch_data.DataLoader(
        valid_data_retriever, 
        batch_size=BATCH_SIZE ,
        shuffle=False,
        num_workers=8 * GPU_NUM,
    )

    model = build_model()
    model.to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
    criterion = torch_functional.binary_cross_entropy_with_logits

    trainer = Trainer(
        model, 
        device, 
        optimizer, 
        criterion
    )

    history = trainer.fit(
        N_EPOCHS, 
        train_loader, 
        valid_loader, 
        f"{mri_type}", 
        N_EPOCHS,
    )

    return trainer.lastmodel

In [11]:
FOLD5 = True
PREDICATION = False

PREDICATION = PREDICATION
def get_modelfiles():
    modelfiles = ['','','',''] * 5
    target_directory = f'./model3d{SEED}/'
    file_list = os.listdir(target_directory)
    best = [0,0,0,0] * 5
    best_name = ['FLAIR','T1w-','T1wCE','T2w'] * 5
    for file in file_list:
            directory = target_directory+file
            for every_file in os.listdir(directory):
                for i in range(0,20):
                    if(best_name[i] in every_file):
                        if(('-FOLD'+str(i//4)) in every_file):
                            best_now = float(every_file.split('auc')[1].split('.pth')[0].split('-')[0])
                            modelfiles[i] = directory + '/' + every_file if best_now > best[i] else modelfiles[i]
                            best[i] = best_now if best_now > best[i] else best[i]

    return modelfiles

if(PREDICATION):
    modelfiles = get_modelfiles()
    print(modelfiles)
if not modelfiles:
    modelfiles = []
    if(FOLD5):
        from sklearn.model_selection import StratifiedKFold
        skf = StratifiedKFold(n_splits=5, random_state=SEED, shuffle=True)
        modelfiles = []
        for n_fold, (train_index, valid_index) in enumerate(skf.split(train_df[["BraTS21ID"]], train_df["MGMT_value"])):
            # if n_fold in [0]: # [0,1,2,3,4]
            FOLD = str(n_fold)
            df_train = train_df.iloc[train_index]
            df_valid = train_df.iloc[valid_index]
            modelfiles.extend([train_mri_type(df_train, df_valid, m) for m in mri_types])
    else:
        n_fold = 0
        FOLD = str(n_fold)
        df_train, df_valid = sk_model_selection.train_test_split(
            train_df, 
            test_size=0.2, 
            random_state=SEED, 
            stratify=train_df["MGMT_value"], 
        )

        modelfiles = [train_mri_type(df_train, df_valid, m) for m in mri_types]
    print(modelfiles)

In [16]:

def predict(modelfile, df, mri_type, split):
    print("Predict:", modelfile, mri_type, df.shape)
    df.loc[:,"MRI_Type"] = mri_type
    data_retriever = Dataset(
        df.index.values, 
        mri_type=df["MRI_Type"].values,
        split=split
    )

    data_loader = torch_data.DataLoader(
        data_retriever,
        batch_size=BATCH_SIZE,
        shuffle=False,
        num_workers=8,
    )
   
    model = build_model()
    model.to(device)
    
    checkpoint = torch.load(modelfile)
    model.load_state_dict(checkpoint["model_state_dict"])
    model.eval()
    
    y_pred = []
    ids = []

    for e, batch in enumerate(data_loader,1):
        print(f"{e}/{len(data_loader)}", end="\r")
        with torch.no_grad():
            tmp_pred = torch.sigmoid(model(batch["X"].float().to(device)).squeeze(1)).cpu().numpy().squeeze()
            if tmp_pred.size == 1:
                y_pred.append(tmp_pred)
            else:
                y_pred.extend(tmp_pred.tolist())
            ids.extend(batch["id"].numpy().tolist())
            
    preddf = pd.DataFrame({"BraTS21ID": ids, "MGMT_value": y_pred}) 
    preddf = preddf.set_index("BraTS21ID")
    return preddf


import time
start = time.time()
submission = pd.read_csv(f"{data_directory}/sample_submission.csv", index_col="BraTS21ID")

submission["MGMT_value"] = 0

import copy
submission_list = []
for i in range(0,5):
    submission_list.append(copy.deepcopy(submission))
    
FOLDN = [0] * 5
for m, mtype in zip(modelfiles, mri_types):
    if(PREDICATION):
        MODEL = m.split('/')[2]
    pred = predict(m, submission, mtype, split="test")
    submission["MGMT_value"] += pred["MGMT_value"]
    for i in range(0,5):
        if(('-FOLD'+str(i)) in m):
            submission_list[i]["MGMT_value"] += pred["MGMT_value"]
            FOLDN[i] += 1
            
for i in range(0,5):
    if(FOLDN[i]!=0):
        submission_list[i] = submission_list[i]/FOLDN[i]
        submission_list[i]["MGMT_value"].to_csv("/submission_"+"-FOLD"+str(i)+".csv")
print(time.time()-start)
submission["MGMT_value"] /= len(modelfiles)
submission["MGMT_value"].to_csv("/submission_"+".csv")
